# Contrastive Representation Learning for Electroencephalogram Classification

Author: Pasquale Ricciulli (2115446)

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import glob
from scipy.signal import butter, filtfilt
from scipy.signal import resample_poly
import librosa
import pickle
from collections import Counter
import warnings
from scipy.io import loadmat
import random
from sklearn.model_selection import train_test_split

The code reduces the size of a preprocessed EEG dataset in .mat (MATLAB) format by applying two main strategies:
___

* **Trial Reduction:** 

    * A random subset of trials is selected (one-third of the total trials per file).

* **Data Reduction:**

    * Number of Channels: Reduced to one-third.
    * Number of Samples: Reduced to half.
The resulting dataset is saved in .npz format (compatible with NumPy).

In [ ]:
#riduzione dataset seed

input_path = '/kaggle/input/seed-dataset/Preprocessed_EEG'
output_path = '/kaggle/working/reduced_dataset'

os.makedirs(output_path, exist_ok=True)

# Parametri per la riduzione
channel_reduction_ratio = 3  # Dividi il numero di canali per 3
sample_reduction_ratio = 2   # Dividi il numero di campioni per 2

mat_files = [f for f in os.listdir(input_path) if f.endswith('.mat') and f != 'label.mat']
print(f"Trovati {len(mat_files)} file .mat nella directory.")

for file_name in mat_files:
    file_path = os.path.join(input_path, file_name)
    print(f"Processando il file: {file_name}")
    
    mat_data = loadmat(file_path)
    
    # Trova tutte le chiavi che rappresentano trial (chiavi con strutture comuni come djc_eeg, ys_eeg, ecc.)
    trial_keys = [key for key in sorted(mat_data.keys()) if not key.startswith('__') and isinstance(mat_data[key], np.ndarray)]
    print(f"Trial trovati in {file_name}: {trial_keys}")
    
    if not trial_keys:
        print(f"Nessun trial trovato in {file_name}, salto il file.")
        continue
    
    # Seleziona casualmente un terzo dei trial
    random.seed(42) 
    selected_keys = random.sample(trial_keys, len(trial_keys) // 3)
    
    for key in selected_keys:  # Itera SOLO sui trial selezionati
        data = mat_data[key]
        
        # Riduzione del numero di canali
        num_channels = data.shape[0]
        reduced_channels = num_channels // channel_reduction_ratio
        reduced_data = data[:reduced_channels, :]
        
        # Riduzione del numero di campioni
        num_samples = reduced_data.shape[1]
        reduced_samples = num_samples // sample_reduction_ratio
        reduced_data = reduced_data[:, :reduced_samples]
        
        output_file = os.path.join(output_path, f"{file_name.replace('.mat', '')}_{key}.npz")
        np.savez(output_file, data=reduced_data)

print(f"Dataset ridotto salvato in: {output_path}")


This code preprocesses EEG signals into `1-second epochs` for further analysis or modeling.
___

**1. Input and Output**

**Input:** `.npz` files containing multichannel EEG signals stored in the `input_folder`.

**Output:** Preprocessed `.npz` files stored in the `output_folder`, with:

* Data divided into 1-second epochs.
* Labels assigned to each file based on a predefined list.
  
**2. Preprocessing Steps**

**File Loading:**

Reads `.npz` files and iterates over each key in the dataset.
Extracts data with shape `(n_channels, n_samples)`.

**Epoch Division:**

**Splits the signal into 1-second epochs:**

`epoch_length = fs = 200` (200 samples per second for a 200 Hz sampling rate).
Computes the total number of complete epochs: `n_epochs = n_samples // epoch_length`.
Each epoch has shape (n_channels, epoch_length).

**Signal Cleaning:**

Removes outliers by setting values above `upper_limit = 500 µV` to 0.

**Label Assignment:**

Uses a predefined list of labels (`labels`) to assign a label to each file.
Label is determined based on the file's index and the modulo of the label list length.

**4. Outputs**
For each input file, the output contains:

Preprocessed data:
Shape: `(n_epochs, n_channels, 200)`.
Assigned label:
Integer representing the file’s category based on the `labels` list.

In [ ]:
#Preprocessamente per epoche da 1 secondo
input_folder = '/kaggle/working/reduced_dataset'
output_folder = '/kaggle/working/preprocessed_1_sec'
os.makedirs(output_folder, exist_ok=True)

# Ignora i warning
warnings.filterwarnings("ignore")

# Parametri per la segmentazione e il filtro
fs = 200  # Frequenza di campionamento in Hz
epoch_length = fs  # Campioni in 1 secondo (200 campioni per 200 Hz)
upper_limit = 500  # Valore massimo per la pulizia del segnale

# Etichette definite
labels = [1, 0, -1, -1, 0, 1, -1, 0, 1, 1, 0, -1, 0, 1, -1]  # 15 etichette

# Funzione per il filtro band-pass Butterworth
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def plot_signals(original_signal, cleaned_signal, channel_idx, file_name):
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 1, 1)
    plt.plot(original_signal, label='Originale', color='blue')
    plt.title(f"{file_name} - Canale {channel_idx} (Originale)")
    plt.xlabel("Campioni")
    plt.ylabel("Ampiezza")
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(cleaned_signal, label='Pulito (±500 µV)', color='red')
    plt.title(f"{file_name} - Canale {channel_idx} (Pulito)")
    plt.xlabel("Campioni")
    plt.ylabel("Ampiezza")
    plt.legend()

    plt.tight_layout()
    plt.show()

file_paths = glob.glob(f"{input_folder}/*.npz")  # File di input

for file_path in file_paths:
    try:
        # Carica il file .npz
        file_name = os.path.basename(file_path).replace('.npz', '')
        print(f"Processando il file: {file_name}")

        npz_data = np.load(file_path)
        
        for key in npz_data.keys():
            data = npz_data[key]  # Forma: (n_channels, n_samples)
            print(f"Elaborazione della chiave: {key} - Forma: {data.shape}")
            
            # Epoche di 1 secondo
            n_channels, n_samples = data.shape
            n_epochs = n_samples // epoch_length  # Numero di epoche complete
            epochs = []

            for i in range(n_epochs):
                start_idx = i * epoch_length
                end_idx = start_idx + epoch_length
                epoch = data[:, start_idx:end_idx]  # Estrai epoca

                # Pulizia del segnale (valori superiori a ±500 µV)
                epoch[epoch > upper_limit] = 0

                epochs.append(epoch)

            epochs = np.array(epochs)  # Forma: (n_epochs, n_channels, epoch_length)
            print(f"Dati suddivisi in epoche per la chiave {key}: {epochs.shape}")

            # Estrazione del numero per la label
            file_number = int(file_name.split('_')[0])  # Prendi il numero del file
            print(file_number)
            label_index = (file_number - 1) % len(labels)
            label = labels[label_index]

            output_path = os.path.join(output_folder, f"processed_{file_name}_{key}.npz")
            np.savez(output_path, data=epochs, label=label)
            print(f"File salvato con etichetta {label} in: {output_path}")

    except Exception as e:
        print(f"Errore durante il preprocessing del file {file_path}: {e}")


This code performs a **split of preprocessed EEG data into training and testing sets**, moving the respective files into separate folders for further use.

**1. Define Input and Output Directories**

**Input Folder:** Preprocessed `.npz` files from the pipeline.

**Output Folders:**
* `train_folder`: Destination for training set files.
* `test_folder`: Destination for testing set files.
  
Ensures both folders exist by creating them if necessary using `os.makedirs`.

**2. Load Preprocessed Files**

Gathers all `.npz` files from the input folder:
```
file_paths = glob.glob(f"{input_folder}/*.npz")
```
Prints the total number of files found.

**3. Split Dataset**

Splits the dataset into **80% training** and **20% testing** using `train_test_split`:
```
train_files, test_files = train_test_split(file_paths, test_size=0.2, random_state=42)
```

Randomization is controlled by `random_state=42` for reproducibility.

**4. Move Files to Train and Test Folders**

A helper function `save_files`:
Iterates through a list of file paths.
Moves each file to the corresponding folder (`train_folder` or `test_folder`).
Uses `os.rename` to transfer files and prints the operation for logging.

**5. Output**

After running, the files are organized as:
```
/kaggle/working/train_set/ -> Contains 80% of the preprocessed files.
/kaggle/working/test_set/  -> Contains 20% of the preprocessed files.
```
Prints confirmation of the split and the number of files in each set.


In [ ]:
#Split del dataset per train e test

input_folder = '/kaggle/working/preprocessed_1_sec'
train_folder = '/kaggle/working/train_set'
test_folder = '/kaggle/working/test_set'
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Ignora i warning
warnings.filterwarnings("ignore")

file_paths = glob.glob(f"{input_folder}/*.npz")
print(f"Trovati {len(file_paths)} file nel dataset elaborato.")

# Suddivisione in train e test
train_files, test_files = train_test_split(file_paths, test_size=0.2, random_state=42)
print(f"File di train: {len(train_files)}, File di test: {len(test_files)}")

# Funzione per spostare i file nelle rispettive cartelle
def save_files(file_list, destination_folder):
    for file_path in file_list:
        file_name = os.path.basename(file_path)
        destination_path = os.path.join(destination_folder, file_name)
        os.rename(file_path, destination_path)
        print(f"File spostato: {file_name} -> {destination_folder}")

save_files(train_files, train_folder)
save_files(test_files, test_folder)

print("Divisione in train e test completata.")


**PREPROCESSING AND REORDERING**

***

In [ ]:
# ------------------- Funzioni per Salvare e Caricare Modelli -------------------
save_path = "/kaggle/working/models"
os.makedirs(save_path, exist_ok=True)

def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Modello salvato in {path}")


#tolgo il module 
def load_model(model, path, device="cuda"):
    state_dict = torch.load(path, map_location=device)
    if "module." in list(state_dict.keys())[0]:
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    print(f"Modello caricato da {path}")

********

This code performs data augmentation and chunking for EEG data, and includes functionality for extracting labels based on file names.
___

#### Augmentation Functions

These functions apply transformations to augment the EEG data, increasing variability and robustness for downstream tasks.

* min_max_amplitude_scale(data, scale_min=0.5, scale_max=2):
  Scales the amplitude of the signal by a random factor between 0.5 and 2.
* time_shift(data, shift_min=-50, shift_max=50):
  Shifts the signal in time by a random number of samples within the range [-50, 50].
* dc_shift(data, shift_min=-10, shift_max=10):
  Adds a constant (DC shift) to the signal, chosen randomly between -10 and 10.
* zero_masking(data, mask_min=0, mask_max=150):
  Masks a random segment of the signal (sets values to 0) with a random length between 0 and 150 samples.
* add_gaussian_noise(data, sigma_min=0, sigma_max=0.2):
  Adds Gaussian noise to the signal, with a standard deviation chosen randomly between 0 and 0.2.

In [ ]:
#AUGMENTATION E CHUNK DATI
def min_max_amplitude_scale(data, scale_min=0.5, scale_max=2):
    scale_factor = random.uniform(scale_min, scale_max)
    return data * scale_factor

def time_shift(data, shift_min=-50, shift_max=50):
    shift_samples = random.randint(shift_min, shift_max)
    return np.roll(data, shift_samples)

def dc_shift(data, shift_min=-10, shift_max=10):
    shift_value = random.uniform(shift_min, shift_max)
    return data + shift_value

def zero_masking(data, mask_min=0, mask_max=150):
    mask_size = random.randint(mask_min, mask_max)
    start_idx = random.randint(0, len(data) - mask_size)
    data[start_idx:start_idx+mask_size] = 0
    return data

def add_gaussian_noise(data, sigma_min=0, sigma_max=0.2):
    sigma = random.uniform(sigma_min, sigma_max)
    noise = np.random.normal(0, sigma, len(data))
    return data + noise

def apply_random_transformations_class(channel_data):
    transformations = [min_max_amplitude_scale, time_shift, dc_shift, zero_masking, add_gaussian_noise]
    selected_transform = random.choice(transformations)
    transformed_data = selected_transform(channel_data.copy())
    return transformed_data
    
def apply_random_transformations(channel_data):
    transformations = [min_max_amplitude_scale, time_shift, dc_shift, zero_masking, add_gaussian_noise]
    selected_transforms = random.sample(transformations, 2)
    transformed_data_1 = selected_transforms[0](channel_data.copy())
    transformed_data_2 = selected_transforms[1](channel_data.copy())
    return transformed_data_1, transformed_data_2
    
    
def chunk_data(data, chunk_size=4000):
    """Divide i dati in chunk della dimensione specificata."""
    chunks = []
    num_chunks = len(data) // chunk_size

    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size
        chunks.append(data[start_idx:end_idx])
    
    return chunks

The provided code implements a **Convolutional Encoder** designed to process sequential data, such as EEG signals.
___
### 1. Constructor (__init__)

The constructor defines the layers and operations of the network. The network consists of three main parts: parallel convolutional paths, repeated blocks, and a final block.

#### Parallel Convolutional Paths
Three convolutional paths operate in parallel, each capturing features at different temporal scales:

Conv1D with kernel size 128:

Adds reflective padding to preserve the original sequence length.
Applies 100 convolutional filters to extract features over a large temporal window.

Conv1D with kernel size 64:

Similar to the first, but with a smaller kernel size, capturing mid-range temporal dependencies.

Conv1D with kernel size 16:

Uses a smaller kernel size to capture fine-grained temporal features.
These parallel paths allow the network to process the signal at multiple scales simultaneously.

#### Dense Layer for Merging

After the parallel paths, their outputs are concatenated and combined through a fully connected (Linear) layer:

Merges the 250 features (100+100+50) into a unified representation.
#### Repeated Blocks

The encoder includes 4 repeated blocks for deeper processing of the signal. Each block consists of:

A ReLU activation for non-linearity.
Batch normalization to stabilize training.
A Conv1D layer with kernel size 64 and 250 filters.
Residual connections are employed: each block adds its input to the output, allowing for efficient gradient flow and preventing vanishing gradients.

#### Final Block

The final layer reduces the dimensionality of the output:

Applies another convolution with a kernel size of 64 and output_dim filters.
ReLU activation and batch normalization ensure stability and non-linearity.
___
### 2. forward: Data Flow
The forward method defines how the data flows through the network.

#### Input

The input is a batch of sequences with the shape:
`
[batch_size, sequence_length, channels]
`

For example, for EEG data with 1 channel, 200 samples per sequence, and a batch size of 32:
`
[32, 200, 1]
`
#### Transpose Input

The input is transposed to match PyTorch's Conv1d requirements:

`x = x.permute(0, 2, 1)`

New shape:

`[batch_size, channels, sequence_length]`

### Parallel Convolutional Paths

The data passes through the three parallel convolutional paths:
```
x1 = self.conv1d_128(x)  # [batch_size, 100, sequence_length]
x2 = self.conv1d_64(x)   # [batch_size, 100, sequence_length]
x3 = self.conv1d_16(x)   # [batch_size, 50, sequence_length]
```
#### Concatenate Outputs

The outputs of the three paths are concatenated along the channel axis:

`x_cat = torch.cat([x1, x2, x3], dim=1)  # [batch_size, 250, sequence_length]`

#### Dense Layer

The concatenated features are passed through a dense layer:

`x_dense = self.concat_dense(x_cat.permute(0, 2, 1)).permute(0, 2, 1)`

This combines features across the temporal axis.

#### Repeated Blocks

The data is passed through the repeated blocks, each adding residual connections:
```
x_repeated = x_dense
for block in self.repeat_blocks:
    x_repeated = x_repeated + block(x_repeated)  # Residual connection
```
#### Final Block

The final block further processes the data to produce the output:
```
x_final = self.final_relu(x_repeated)
x_final = self.final_bn(x_final)
x_final = self.final_conv(x_final)
```
The output is transposed back to match the input format:

`x_final = x_final.permute(0, 2, 1)`

Final shape:

`[batch_size, sequence_length, output_dim]`
___
### 3. Output
The network outputs a tensor with the shape:

```[batch_size, sequence_length, output_dim]```
For example, if:

`batch_size = 32`
`sequence_length = 200`
`output_dim = 4`

The output shape will be:

`[32, 200, 4]`
Each sample in the sequence is represented by a vector of size output_dim.




In [ ]:
class ConvolutionalEncoder(nn.Module):
    def __init__(self, input_channels=1, output_dim=4, repeat_blocks=4):
        super(ConvolutionalEncoder, self).__init__()
        
        # Parallel convolutional paths
        self.conv1d_128 = nn.Sequential(
            nn.ReflectionPad1d((63, 64)),
            nn.Conv1d(input_channels, 100, kernel_size=128, stride=1)
        )
        self.conv1d_64 = nn.Sequential(
            nn.ReflectionPad1d((31, 32)),
            nn.Conv1d(input_channels, 100, kernel_size=64, stride=1)
        )
        self.conv1d_16 = nn.Sequential(
            nn.ReflectionPad1d((7, 8)),
            nn.Conv1d(input_channels, 50, kernel_size=16, stride=1)
        )

        # Dense layer to merge paths
        self.concat_dense = nn.Linear(100 + 100 + 50, 250)

        # Repeat N=4 blocks
        self.repeat_blocks = nn.ModuleList([
            nn.Sequential(
                nn.ReLU(),
                nn.BatchNorm1d(250),
                nn.ReflectionPad1d((31, 32)),
                nn.Conv1d(250, 250, kernel_size=64, stride=1)
            ) for _ in range(repeat_blocks)
        ])

        # Final block
        self.final_relu = nn.ReLU()
        self.final_bn = nn.BatchNorm1d(250)
        self.final_conv = nn.Sequential(
            nn.ReflectionPad1d((31, 32)),
            nn.Conv1d(250, output_dim, kernel_size=64, stride=1)
        )
        
    def forward(self, x):
        # Input shape: [batch_size, sequence_length, channels] modify

        
        x = x.permute(0, 2, 1)  #[batch_size, channels, sequence_length]
        
        # Parallel convolutional paths
        x1 = self.conv1d_128(x)
        x2 = self.conv1d_64(x)
        x3 = self.conv1d_16(x)

        # Concatenate paths
        x_cat = torch.cat([x1, x2, x3], dim=1)  # [batch_size, 250, sequence_length]
        
        # Dense layer
        x_dense = self.concat_dense(x_cat.permute(0, 2, 1)).permute(0, 2, 1)

        # Repeated blocks
        x_repeated = x_dense
        for block in self.repeat_blocks:
            x_repeated = x_repeated + block(x_repeated)  # Residual connection

        # Final block
        x_final = self.final_relu(x_repeated)
        x_final = self.final_bn(x_final)
        x_final = self.final_conv(x_final)
        x_final = x_final.permute(0, 2, 1)
        #print("xfinal:", x_final.shape)
        return x_final  # [batch_size, output_dim]

The **RecurrentEncoder** is a hierarchical recurrent neural network designed to process sequential data, progressively extract features, and combine hierarchical representations. It incorporates GRU layers and a residual recurrent mechanism for deeper temporal learning

___

#### 1. Hierarchical GRU Layers:

The architecture uses three GRU layers at different levels of abstraction:
* **GRU (256):** Processes the input sequence at the highest dimensionality.
* **GRU (128):** Processes downsampled features for mid-level abstraction.
* **GRU (64):** Processes the most downsampled features for fine-grained details.
  
These layers extract temporal dependencies across different feature representations.

#### 2. Downsampling and Upsampling:

**Downsampling:**

Linear layers reduce the dimensionality between consecutive GRU layers (256 → 128 → 64).
Helps in compressing features for more efficient learning.

**Upsampling:**

Linear layers increase the dimensionality back (64 → 128 → 256) to unify features at different levels.

#### 3. Feature Concatenation:

Features from all GRU layers are concatenated along the feature axis to create a comprehensive representation:

* `[x_256, x_128, x_64]`.
This combines information from all levels of the hierarchy.

#### 4. Recurrent Residual Units (RRU):

A series of GRU layers with residual connections are applied to the concatenated features.
Residual connections help in preserving the original information while learning additional temporal dependencies.

Each RRU consists of:

A layer normalization for stability.
A GRU to model temporal dynamics.
5. Output Layer:

A fully connected layer maps the processed hidden representation to the final `output_dim`.
___
### Data Flow (forward method)
**Input:**
The input x is a sequence with shape:

`[batch_size, sequence_length, input_dim]`

**Hierarchical Feature Extraction:**

* Data flows through the hierarchical GRU layers:
  * `x_256` (GRU with 256 units).
  * Downsampled to `x_128_input`, then passed to GRU (128 units).
  * Downsampled to `x_64_input`, then passed to GRU (64 units).
* Each GRU captures temporal dependencies at its respective level.

**Upsampling:**

The output from the lowest GRU level (`x_64`) is progressively upsampled to match higher-level dimensions.

**Feature Concatenation:**

Combine the outputs from all GRU levels (`x_256`, `x_128`, `x_64`) into a unified representation.

**Residual Recurrent Processing:**

The concatenated features pass through a series of **residual GRU units** (RRU layers) to refine temporal relationships.

**Output:**
The final representation is passed through a dense layer to map it to the desired `output_dim`.

Output shape:

`[batch_size, sequence_length, output_dim]`


In [ ]:
class RecurrentEncoder(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, repeat_n=2):
        super(RecurrentEncoder, self).__init__()

        self.gru_256 = nn.GRU(input_dim, 256, batch_first=True)
        self.downsample_256_128 = nn.Linear(256, 128)
        self.gru_128 = nn.GRU(128, 128, batch_first=True)
        self.downsample_128_64 = nn.Linear(128, 64)
        self.gru_64 = nn.GRU(64, 64, batch_first=True)

        self.upsample_64_128 = nn.Linear(64, 128)
        self.upsample_128_256 = nn.Linear(128, 256)

        self.concat_dense = nn.Linear(256 + 128 + 64, hidden_dim)

        self.rru = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm(hidden_dim),
                nn.GRU(hidden_dim, hidden_dim, batch_first=True)
            ) for _ in range(repeat_n)
        ])

        self.output_dense = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x_256, _ = self.gru_256(x)
        x_128_input = F.relu(self.downsample_256_128(x_256))
        x_128, _ = self.gru_128(x_128_input)
        x_64_input = F.relu(self.downsample_128_64(x_128))
        x_64, _ = self.gru_64(x_64_input)

        x_128_up = F.relu(self.upsample_64_128(x_64))
        x_256_up = F.relu(self.upsample_128_256(x_128_up))

        x_concat = torch.cat([x_256, x_128, x_64], dim=-1)
        x_hidden = F.relu(self.concat_dense(x_concat))

        for i, rru_layer in enumerate(self.rru):
            residual, _ = rru_layer(x_hidden)
            x_hidden = x_hidden + residual

        output = self.output_dense(x_hidden)
        return output


*****


**1. EEGClassificationDataset Class**

A PyTorch Dataset class for loading and preprocessing EEG data for classification tasks.

**Purpose:**
* Loads `.npz` files containing EEG data.
* Remaps labels (`-1, 0, 1` → `0, 1, 2`) for multi-class classification.
* Transposes data to have the shape `(num_samples, seq_len, num_channels)`.

**Output:**
Returns a sequence (`data`) and its corresponding label for training.

**2. Classifier Class**
A neural network for classifying EEG data using hierarchical Bidirectional LSTMs (BiLSTMs).

Architecture:
Three stages of dimensionality reduction using `Linear` layers followed by BiLSTMs:
* `input_dim → 256 → 128 → 64`.
  
* Final dense layers combine features and produce classification logits.
* Uses `LogSoftmax` for probabilistic outputs.


**Output:**

Log-probabilities for each class.

**3. load_model and save_model Functions**

* **Purpose:** Save and load PyTorch models.
* Handles models trained with `DataParallel` by removing the `module.` prefix from saved state dictionaries.
  
**4. train_classifier Function**

The training loop for optimizing the classifier.

**Workflow:**

* **Dataset and Dataloader:**
  
Loads EEG data using EEGClassificationDataset.
Creates batches of data with a specified batch size.

**Encoder Freeze:**

Freezes the pretrained encoder to prevent updates during classifier training.

**Forward Pass:**

Processes each channel of the data through the encoder (pretrained).

Concatenates embeddings from all channels.

Passes concatenated embeddings through the classifier.

**Loss and Optimization:**

Uses Negative Log-Likelihood Loss (NLLLoss) for classification.
Optimizes the classifier using backpropagation.

**Metrics and Logging:**

Tracks the loss and accuracy per batch and epoch.
Logs intermediate metrics for monitoring.

**Model Checkpoints:**

Saves the classifier model at the end of each epoch for reproducibility.

In [ ]:
class EEGClassificationDataset(Dataset):
    def __init__(self, file_paths, seq_len=200):
        self.samples = []
        self.labels = []
        self.seq_len = seq_len

        for file_path in file_paths:
            npz_data = np.load(file_path)
            data = npz_data['data']  # (num_samples, num_channels, seq_len)
            label = npz_data['label']  # scalare

            if isinstance(label, np.ndarray):
                label = label.item()

            # label potrebbe essere -1, 0 o 1. Li rimappiamo in 0, 1, 2.
            if label == -1:
                label = 0
            elif label == 0:
                label = 1
            elif label == 1:
                label = 2

            labels = np.full((data.shape[0],), label)

            # Cambia la forma a (num_samples, seq_len, num_channels)
            data = data.transpose(0, 2, 1)

            self.samples.extend(data)
            self.labels.extend(labels)

        class_counts = Counter(self.labels)
        print("Distribuzione delle classi nel dataset:", class_counts)
        print("Dataset creato con", len(self.samples), "segmenti.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        data = torch.tensor(self.samples[idx], dtype=torch.float32)  # (seq_len, num_channels)
        label = int(self.labels[idx])
        return data, label


class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()

        self.downsample_1 = nn.Linear(input_dim, 256)
        self.bilstm_256 = nn.LSTM(256, 128, batch_first=True, bidirectional=True)

        self.downsample_2 = nn.Linear(256, 128)
        self.bilstm_128 = nn.LSTM(128, 64, batch_first=True, bidirectional=True)

        self.downsample_3 = nn.Linear(128, 64)
        self.bilstm_64 = nn.LSTM(64, 32, batch_first=True, bidirectional=True)

        self.concat_dense_1 = nn.Linear(256 + 128 + 64, 128)
        self.concat_dense_2 = nn.Linear(128, num_classes)

        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x_256 = F.relu(self.downsample_1(x))
        x_256, (h_256, _) = self.bilstm_256(x_256)
        flo_256 = torch.cat([h_256[0], h_256[1]], dim=-1)

        x_128 = F.relu(self.downsample_2(x_256))
        x_128, (h_128, _) = self.bilstm_128(x_128)
        flo_128 = torch.cat([h_128[0], h_128[1]], dim=-1)

        x_64 = F.relu(self.downsample_3(x_128))
        x_64, (h_64, _) = self.bilstm_64(x_64)
        flo_64 = torch.cat([h_64[0], h_64[1]], dim=-1)

        x_concat = torch.cat([flo_256, flo_128, flo_64], dim=-1)
        x_hidden = F.relu(self.concat_dense_1(x_concat))
        output = self.concat_dense_2(x_hidden)
        return self.log_softmax(output)

def load_model(model, path, device="cuda"):
    state_dict = torch.load(path, map_location=device)
    # Rimuoviamo eventuale prefisso 'module.' se salvato con DataParallel
    if any(k.startswith('module.') for k in state_dict.keys()):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    model.to(device)

def save_model(model, path):
    if isinstance(model, nn.DataParallel):
        model = model.module
    torch.save(model.state_dict(), path)

def train_classifier(model_type, file_paths, encoder, classifier, optimizer, epochs=1, batch_size=40, device="cuda"):
    dataset = EEGClassificationDataset(file_paths)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    encoder.eval()
    for param in encoder.parameters():
        param.requires_grad = False
    encoder = nn.DataParallel(encoder).to(device)

    classifier = nn.DataParallel(classifier).to(device)
    classifier.train()
    loss_fn = nn.NLLLoss()

    save_path = "./model_checkpoints"
    os.makedirs(save_path, exist_ok=True)

    for epoch in range(epochs):
        print(f"=== Inizio Epoca {epoch + 1}/{epochs} ===")
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        for batch_idx, (data, labels) in enumerate(dataloader):
            #print(f"Batch {batch_idx + 1}: Shape data: {data.shape}, Shape labels: {labels.shape}")
            data, labels = data.to(device), labels.to(device)

            batch_outputs = []
            # Itera sui canali
            for channel_idx in range(data.shape[2]):
                channel_data = data[:, :, channel_idx].unsqueeze(-1)  # (batch_size, seq_len, 1)
                with torch.no_grad():
                    channel_output = encoder(channel_data)  # (batch_size, seq_len, 4)
                batch_outputs.append(channel_output)

            # Concateniamo le feature di tutti i canali
            embeddings = torch.cat(batch_outputs, dim=-1)  # (batch_size, seq_len, 4*num_channels)
            #print("Embeddings shape:", embeddings.shape)

            logits = classifier(embeddings)
            loss = loss_fn(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

            if batch_idx % 10 == 0:
                accuracy = 100 * correct_predictions / total_predictions
                print(f"Epoca {epoch + 1}/{epochs}, Batch {batch_idx + 1}/{len(dataloader)}, Perdita: {loss.item():.4f}, Accuratezza: {accuracy:.2f}%")

        epoch_accuracy = 100 * correct_predictions / total_predictions
        print(f"Epoca {epoch + 1}/{epochs}, Perdita Totale: {total_loss:.4f}, Accuratezza: {epoch_accuracy:.2f}%")
        save_model(classifier, os.path.join(save_path, f"classifier_{model_type}_epoch_{epoch + 1}.pt"))


In [ ]:
if __name__ == "__main__":
    folder_path = "/kaggle/working/train_set"
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.npz')]

    encoder = RecurrentEncoder(input_dim=1, output_dim=4)
    load_model(encoder, "/kaggle/input/recurrent_encoder_/pytorch/default/1/Recurrent_epoch_24.pth", device="cuda")

    # Il numero di canali è 20 e l'encoder produce 4 feature per canale: input_dim=4*num_channels=4*20=80
    classifier = Classifier(input_dim=4 * 20, num_classes=3)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

    train_classifier("Recurrent", file_paths, encoder, classifier, optimizer, epochs=10, batch_size=40, device="cuda")

In [ ]:
if __name__ == "__main__":
    folder_path = "/kaggle/working/train_set"
    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.npz')]

    encoder = ConvolutionalEncoder(input_channels=1, output_dim=4)
    load_model(encoder, "/kaggle/input/convolutional_encoder/pytorch/default/1/Convolutional_epoch_10.pth", device="cuda")

    # Il numero di canali è 20 e l'encoder produce 4 feature per canale: input_dim=4*num_channels=4*20=80
    classifier = Classifier(input_dim=4 * 20, num_classes=3)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

    train_classifier("Convolutional", file_paths, encoder, classifier, optimizer, epochs=10, batch_size=40, device="cuda")

**EVALUATION**

**1. EEGEvaluationDataset Class**

This PyTorch Dataset class prepares EEG test data for evaluation.

**Key Features:**

Loads and processes EEG data:
Reads `.npz` files containing EEG data (`data`) and their associated labels (`label`).

**Label remapping:**

* Remaps labels:
* `-1 → 0` (negative).
* `0 → 1` (neutral).
* `1 → 2` (positive).
* 
**Transposes data:**
  
Converts data shape from `(num_samples, num_channels, seq_len)` to `(num_samples, seq_len, num_channels)` for compatibility with PyTorch models.
**Output:**

Each sample (`data`) and its label are returned during iteration.

**2. Classifier Class**

A neural network for EEG signal classification.

**Architecture:**

**Hierarchical BiLSTM Blocks:**

* Three stages:

* Dimensionality reduction with `Linear` layers.
* Temporal feature extraction with bidirectional LSTMs (BiLSTMs).
* Dimensions reduced as: `input_dim → 256 → 128 → 64`.

**Feature Concatenation:**
Combines outputs from all three stages into a single feature vector.

**Final Classification:**

Dense layers map the concatenated features to class logits.
LogSoftmax provides probabilistic outputs for classification.

**Output:**

Log-probabilities for each class.

**3. Utility Functions**

**load_model:**

* Loads a pre-trained model from a file.

* Removes any module. prefixes in the state dictionary for compatibility with single/multi-GPU setups.

**evaluate_classifier:**

Evaluates the encoder and classifier on a test dataset.

**4. evaluate_classifier Function**

The main function that evaluates the classifier using the test dataset.

**Workflow:**

**Dataset and Dataloader:**

* Loads EEG test data using the `EEGEvaluationDataset` class.
* Creates a dataloader to iterate through the test samples.
**Model Evaluation:**
  
* Sets both the encoder and classifier to evaluation mode (`eval()`).
* Freezes gradients (`torch.no_grad()`).

**Forward Pass:**

Processes data through the encoder for each channel:
* Each channel is passed individually through the encoder.
* Channel-wise embeddings are concatenated.
The concatenated embeddings are passed through the classifier to generate logits.

**Metrics Calculation:**

Predictions (`all_preds`) and true labels (`all_labels`) are stored.
Computes accuracy, confusion matrix, and classification report.
**5. Outputs**

**Accuracy:**

Overall accuracy of the classifier on the test set.

**Confusion Matrix:**

Displays the number of true positives, false positives, and false negatives for each class.

**Classification Report:**

Precision, recall, and F1-score for each class.


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

class EEGEvaluationDataset(Dataset):
    def __init__(self, file_paths, seq_len=200):
        self.samples = []
        self.labels = []
        self.seq_len = seq_len

        for file_path in file_paths:
            npz_data = np.load(file_path)
            data = npz_data['data']  # (num_samples, num_channels, seq_len)
            label = npz_data['label']  

            if isinstance(label, np.ndarray):
                label = label.item()

            if label == -1:
                label = 0
            elif label == 0:
                label = 1
            elif label == 1:
                label = 2

            labels = np.full((data.shape[0],), label)

            # Cambia forma a (num_samples, seq_len, num_channels)
            data = data.transpose(0, 2, 1)

            self.samples.extend(data)
            self.labels.extend(labels)

        class_counts = Counter(self.labels)
        print("Distribuzione delle classi nel dataset (test):", class_counts)
        print("Test set creato con", len(self.samples), "segmenti.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        data = torch.tensor(self.samples[idx], dtype=torch.float32)  # (seq_len, num_channels)
        label = int(self.labels[idx])
        return data, label

class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(Classifier, self).__init__()

        self.downsample_1 = nn.Linear(input_dim, 256)
        self.bilstm_256 = nn.LSTM(256, 128, batch_first=True, bidirectional=True)

        self.downsample_2 = nn.Linear(256, 128)
        self.bilstm_128 = nn.LSTM(128, 64, batch_first=True, bidirectional=True)

        self.downsample_3 = nn.Linear(128, 64)
        self.bilstm_64 = nn.LSTM(64, 32, batch_first=True, bidirectional=True)

        self.concat_dense_1 = nn.Linear(256 + 128 + 64, 128)
        self.concat_dense_2 = nn.Linear(128, num_classes)

        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x_256 = F.relu(self.downsample_1(x))
        x_256, (h_256, _) = self.bilstm_256(x_256)
        flo_256 = torch.cat([h_256[0], h_256[1]], dim=-1)

        x_128 = F.relu(self.downsample_2(x_256))
        x_128, (h_128, _) = self.bilstm_128(x_128)
        flo_128 = torch.cat([h_128[0], h_128[1]], dim=-1)

        x_64 = F.relu(self.downsample_3(x_128))
        x_64, (h_64, _) = self.bilstm_64(x_64)
        flo_64 = torch.cat([h_64[0], h_64[1]], dim=-1)

        x_concat = torch.cat([flo_256, flo_128, flo_64], dim=-1)
        x_hidden = F.relu(self.concat_dense_1(x_concat))
        output = self.concat_dense_2(x_hidden)
        return self.log_softmax(output)

def load_model(model, path, device="cuda"):
    state_dict = torch.load(path, map_location=device)
    if any(k.startswith('module.') for k in state_dict.keys()):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    model.to(device)

def evaluate_classifier(file_paths, encoder, classifier,batch_size=40, device="cuda"):
    dataset = EEGEvaluationDataset(file_paths)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    encoder.eval()
    classifier.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)

            batch_outputs = []
            for channel_idx in range(data.shape[2]):
                channel_data = data[:, :, channel_idx].unsqueeze(-1)  # (batch_size, seq_len, 1)
                channel_output = encoder(channel_data)  # (batch_size, seq_len, 4)
                batch_outputs.append(channel_output)

            embeddings = torch.cat(batch_outputs, dim=-1)  # (batch_size, seq_len, 4*num_channels)
            logits = classifier(embeddings)
            _, predicted = torch.max(logits, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calcolo metriche
    acc = accuracy_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)

    print("=== Risultati sul Test Set ===")
    print("Accuratezza:", acc)
    print("Matrice di Confusione:")
    print(cm)
    print("Report di Classificazione (Precision, Recall, F1-Score):")
    print(report)


In [ ]:
if __name__ == "__main__":
    test_folder_path = "/kaggle/working/test_set"
    test_file_paths = [os.path.join(test_folder_path, f) for f in os.listdir(test_folder_path) if f.endswith('.npz')]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"


    encoder = RecurrentEncoder(input_dim=1, output_dim=4)  
    classifier = Classifier(input_dim=4 * 20, num_classes=3)

    load_model(encoder, "/kaggle/input/recurrent_encoder_/pytorch/default/1/Recurrent_epoch_24.pth", device=device)
    load_model(classifier, "/kaggle/input/classifier_recurrent/pytorch/default/1/classifier_Recurrent_epoch_10.pt", device=device)

    evaluate_classifier(test_file_paths, encoder, classifier,batch_size=40, device=device)

In [ ]:
if __name__ == "__main__":
    test_folder_path = "/kaggle/working/test_set"
    test_file_paths = [os.path.join(test_folder_path, f) for f in os.listdir(test_folder_path) if f.endswith('.npz')]
    
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Carica i modelli già addestrati
    encoder = ConvolutionalEncoder(input_channels=1, output_dim=4) 
    classifier = Classifier(input_dim=4 * 20, num_classes=3)  

    load_model(encoder, "/kaggle/input/convolutional_encoder/pytorch/default/1/Convolutional_epoch_10.pth", device=device)
    load_model(classifier, "/kaggle/input/classifier_convolutional/pytorch/default/1/classifier_Convolutional_epoch_10.pt", device=device)

    evaluate_classifier(test_file_paths, encoder, classifier,batch_size=40, device=device)